# Sequence generation

- all available historical data
- include static features

- attention mechanism

- validation on DataLoch

- model explanation

In [1]:
import pandas as pd
import pyreadr
import numpy as np
import torch
import pickle
from os import listdir
from os.path import isfile, join

In [ ]:
patient = pyreadr.read_r('../ServerData_13Oct2020/d_patient_overall.Rdata')
practice = pyreadr.read_r('../ServerData_13Oct2020/d_practice.Rdata')
patient = patient['d_patient_overall']
practice = practice['d_practice']

#Age in 2016-01-01
patient['age'] = patient.year_of_birth.apply(lambda x: 2016-x)

patient = patient[['patid', 'practice_id', 'age']].merge(practice[['practice_id', 'Country']], how='left', on='practice_id')

# Outcomes data
outcomes = pd.read_csv("../FinalData/cleaned_outcomes_11072023.csv")

In [ ]:
outcomes['new_12MonthsOutcome'] = outcomes.apply(lambda x: (x.outcome_15months)|(x.outcome_18months)|(x.outcome_21months)|(x.outcome_24months), axis=1)

In [ ]:
data_all_big = pickle.load(open('../SeqModel/data_all_big0.sav', 'rb'))
vocab_all_big = pickle.load(open('../SeqModel/vocab_all_big0.sav', 'rb'))
for i in range(1,3):
    temp_data = pickle.load(open('../SeqModel/data_all_big'+str(i)+'.sav', 'rb'))
    temp_vocab = pickle.load(open('../SeqModel/vocab_all_big'+str(i)+'.sav', 'rb'))
    data_all_big = pd.concat([data_all_big, temp_data])
    vocab_all_big = vocab_all_big + temp_vocab
    
    
#vocab and code2idx generation
# vocab_all_big = vocab_all
vocab_all_big=list(dict.fromkeys(vocab_all_big)) #remove duplicate
idx_all_big = range(3, len(vocab_all_big)+3)
code2idx_all_big = dict(zip(vocab_all_big, idx_all_big))
idx2code_all_big = dict(zip(idx_all_big, vocab_all_big))
code2idx_all_big['PAD'] = 0
code2idx_all_big['start_visit'] = 1
code2idx_all_big['end_visit'] = 2
idx2code_all_big[0] = 'PAD'
idx2code_all_big[1] = 'start_visit'
idx2code_all_big[2] = 'end_visit'

VOCAB_SIZE_big = len(code2idx_all_big)
print('code2idx Size: {}'.format(len(code2idx_all_big)))
print('idx2code Size: {}'.format(len(idx2code_all_big)))
# print(code2idx)

# padded to index, and merge with patient data (age, Country)
# data_all_big = data_all
data_all_big['read_code_seq_padded_idx'] = data_all_big['read_code_seq_padded'].apply(lambda x: [code2idx_all_big.get(key) for key in x])
data_all_big = data_all_big.merge(outcomes[['patid', 'outcome_3months','outcome_combined_6months','outcome_combined_12months',
                               'outcome_combined_24months', 'new_12MonthsOutcome']], how='inner', on='patid')
data_all_big = data_all_big.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
data_all_big = data_all_big.drop_duplicates(subset=['patid']).reset_index(drop=True)      

print(data_all_big.shape)

In [ ]:
# #load clinical information
# path = '../ServerData_13Oct2020/'
# clinical_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_clinical_part' in f)]
# clinical = pyreadr.read_r('../ServerData_13Oct2020/f_clinical_part1.Rdata')
# clinical = clinical['f_clinical_part']
# for file in clinical_files[1:3]:
#     temp = pyreadr.read_r(file)
#     temp = temp['f_clinical_part']
#     print(temp.shape)
#     clinical = pd.concat([clinical, temp])
#     clinical.reset_index(drop=True, inplace=True)
    
# #data selection 
# clinical = clinical.dropna(subset=['code_id'])
# clinical['event_date'] = pd.to_datetime(clinical['event_date'])
# clinical = clinical.loc[(clinical['event_date'] >= '2016-01-01') & (clinical['event_date'] < '2018-01-01')]

# #add paractice information

# #load clinical information
# path = '../ServerData_13Oct2020/'
# therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
# therapy = pyreadr.read_r('../ServerData_13Oct2020/f_therapy_part1.Rdata')
# therapy = therapy['f_therapy_part']
# for file in therapy_files[1:3]:
#     temp = pyreadr.read_r(file)
#     temp = temp['f_therapy_part']
#     print(temp.shape)
#     therapy = pd.concat([therapy, temp])
#     therapy.reset_index(drop=True, inplace=True)
    
# #data selection 
# therapy = therapy.dropna(subset=['code_id'])
# therapy['event_date'] = pd.to_datetime(therapy['event_date'])
# therapy = therapy.loc[(therapy['event_date'] >= '2016-01-01') & (therapy['event_date'] < '2018-01-01')]

In [ ]:
# all_raw_data = pd.concat([clinical[['patid', 'event_date', 'code_id']],
#            therapy[['patid', 'event_date', 'code_id']]])
# all_raw_data.reset_index(drop=True, inplace=True)
# #vocab and code2idx generation
# vocab_all = all_raw_data.code_id.unique().tolist()
# idx_all = range(1, len(vocab_all)+1)

# code2idx_all = dict(zip(vocab_all, idx_all))
# idx2code_all = dict(zip(idx_all, vocab_all))

# code2idx_all['PAD'] = 0
# code2idx_all['start_visit'] = 1
# code2idx_all['end_visit'] = 2
# idx2code_all[0] = 'PAD'
# idx2code_all[1] = 'start_visit'
# idx2code_all[2] = 'end_visit'
# VOCAB_SIZE= len(code2idx_all)
# print('code2idx Size: {}'.format(len(code2idx_all)))
# print('idx2code Size: {}'.format(len(idx2code_all)))
# # print(code2idx)
# #extract year, month, day from event date
# all_raw_data['day'] = all_raw_data.apply(lambda x: str(x['event_date'].day), axis=1)
# all_raw_data['month'] = all_raw_data.apply(lambda x: str(x['event_date'].month), axis=1)
# all_raw_data['year'] = all_raw_data.apply(lambda x: str(x['event_date'].year), axis=1)
# all_raw_data.reset_index(drop=True, inplace=True)
# all_raw_data['read_code_seq_perdate'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby(['patid', 'event_date'])['code_id'].transform(lambda x: ', '.join(x))
# all_raw_data['read_code_seq_perdate'] = all_raw_data['read_code_seq_perdate'].apply(lambda x: 'start_visit, ' + x + ', end_visit')
# all_raw_data = all_raw_data.drop_duplicates(subset=['patid', 'event_date']).reset_index(drop=True)
# all_raw_data['read_code_seq'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby(['patid'])['read_code_seq_perdate'].transform(lambda x: ', '.join(x))
# # all_raw_data['read_code_seq'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
# event_data_seq_all = all_raw_data.sort_values(['event_date']).groupby('patid').agg({'day': lambda x: x.tolist(),
#                                                           'month': lambda x: x.tolist(),
#                                                           'year': lambda x: x.tolist()}).reset_index()
# all_raw_data = all_raw_data.drop_duplicates(subset=['patid']).reset_index(drop=True)     
# data_all = all_raw_data[['patid', 'read_code_seq']].merge(event_data_seq_all, how='left',  on='patid')
# data_all = data_all.merge(outcomes[['patid', 'new_12MonthsOutcome',
#                            'outcome_combined_24months']], how='inner', on='patid')

# data_all['read_code_seq'] = data_all['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
# data_all['length_read_code_seq'] = data_all['read_code_seq'].apply(lambda x: len(x))
# data_all = data_all[data_all.length_read_code_seq > 10]

# #padding at the beginning of the list
# max_seq = 300
# def make_uniform_data(x):
#     if len(x) < max_seq:
#         pads = ['PAD'] * (max_seq - len(x))
#         return pads + x
#     elif len(x) > max_seq:
#         x = x[len(x)-max_seq:]
#         return x
#     else:
#         return x
    
# data_all['read_code_seq_padded'] = data_all['read_code_seq'].apply(lambda x: make_uniform_data(x))
# data_all['read_code_seq_padded_idx'] = data_all['read_code_seq_padded'].apply(lambda x: [code2idx_all.get(key) for key in x])
# data_all = data_all.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
# data_all = data_all.drop_duplicates(subset=['patid']).reset_index(drop=True)     

# #SAVE all important materials
# pickle.dump(code2idx_all, open('../SeqModel/code2idx_all_new.sav', 'wb'))
# pickle.dump(idx2code_all, open('../SeqModel/idx2code_all_new.sav', 'wb'))
# pickle.dump(data_all, open('../SeqModel/data_all_new.sav', 'wb'))
# # data_all.to_csv('../SeqModel/seqData_all.csv', index_label=False, index=False)

# LSTM Model using Clinical + Therapy sequence

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, SensitivityAtSpecificity
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop, Adamax, SGD
from tensorflow.keras.initializers import Constant
from tensorflow.keras.regularizers import L1L2, L1, L2
from livelossplot import PlotLossesKeras

# from tensorflow.keras.backend.tensorflow_backend import set_session
from tensorflow.python.keras import backend as K

#internal validation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, balanced_accuracy_score, matthews_corrcoef, auc, average_precision_score, roc_auc_score, balanced_accuracy_score, roc_curve, accuracy_score

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pickle

# fix random seed for reproducibility
tf.random.set_seed(1234)

target_outcome = 'new_12MonthsOutcome'
max_codes = 300

2023-11-16 13:49:40.917881: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 13:49:41.217307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 13:49:41.217361: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 13:49:41.219110: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 13:49:41.374685: I tensorflow/core/platform/cpu_feature_g

In [ ]:
# load the dataset but only keep the top n words, zero the rest
    
code2idx = pickle.load(open('../SeqModel/code2idx_all_big.sav', 'rb'))
idx2code = pickle.load(open('../SeqModel/idx2code_all_big.sav', 'rb'))
data = pickle.load(open('../SeqModel/data_all_big.sav', 'rb'))

vocab_size = len(code2idx)
print(vocab_size)
print(data.shape)

In [ ]:
# #Data split conventional (mixed countries)
# trainingData, testData = train_test_split(data, test_size=0.1, stratify=data[target_outcome], random_state=1234)
# trainingData, valData = train_test_split(trainingData, test_size=0.2, stratify=trainingData[target_outcome], random_state=1234)
# print(trainingData.shape)
# print(valData.shape)
# print(testData.shape)

In [ ]:
#Data split, train=England, eval=Scot+Wales
# data = data_all_big
# data_all_big = [] #free up the memory
vocab_size = len(code2idx)
trainingData = data[(data.Country == 'England') & (data.age >= 18)]
trainingData, valData = train_test_split(trainingData, test_size=0.3, stratify=trainingData[target_outcome], shuffle=True, random_state=1234)
trainingData, evalData = train_test_split(trainingData, test_size=0.2, stratify=trainingData[target_outcome], shuffle=True, random_state=1234)
testData = data[((data.Country == 'Wales') | (data.Country == 'Scotland')) & (data.age >= 18)]
testDataWales = data[(data.Country == 'Wales') & (data.age >= 18)]
testDataScotland = data[(data.Country == 'Scotland') & (data.age >= 18)]

In [ ]:
print('Train: ', trainingData.shape[0])
print('Val: ', valData.shape[0])
print('Eval (internal validation): ', evalData.shape[0])
print('Test: ', testData.shape[0])
print('Test - Wales: ', testDataWales.shape[0])
print('Test - Scotland: ', testDataScotland.shape[0])

In [ ]:
print(trainingData.patid.unique().shape)
print(trainingData.shape)

In [ ]:
#make sure no data leak between sets
print(list(set(trainingData.patid.values).intersection(set(valData.patid.values))))
print(list(set(trainingData.patid.values).intersection(set(evalData.patid.values))))
print(list(set(valData.patid.values).intersection(set(evalData.patid.values))))
print(list(set(valData.patid.values).intersection(set(testData.patid.values))))
print(list(set(trainingData.patid.values).intersection(set(testData.patid.values))))
print(len(list(set(testData.patid.values).intersection(set(testDataScotland.patid.values))))) # here data leak is expected)

In [ ]:
print(trainingData[target_outcome].value_counts(normalize=True))
print(valData[target_outcome].value_counts(normalize=True))
print(evalData[target_outcome].value_counts(normalize=True))
print(testData[target_outcome].value_counts(normalize=True))
print(testDataWales[target_outcome].value_counts(normalize=True))
print(testDataScotland[target_outcome].value_counts(normalize=True))

In [ ]:
# X and y
X_train = np.array(trainingData.read_code_seq_padded_idx.values)
X_train = np.array([x for x in X_train])
X_val = np.array(valData.read_code_seq_padded_idx.values)
X_val = np.array([x for x in X_val])
X_eval = np.array(evalData.read_code_seq_padded_idx.values)
X_eval = np.array([x for x in X_eval])
X_test = np.array(testData.read_code_seq_padded_idx.values)
X_test = np.array([x for x in X_test])
X_testWales = np.array(testDataWales.read_code_seq_padded_idx.values)
X_testWales = np.array([x for x in X_testWales])
X_testScotland = np.array(testDataScotland.read_code_seq_padded_idx.values)
X_testScotland = np.array([x for x in X_testScotland])

y_train = trainingData[target_outcome].values
y_val = valData[target_outcome].values
y_eval = evalData[target_outcome].values
y_test = testData[target_outcome].values
y_testWales = testDataWales[target_outcome].values
y_testScotland = testDataScotland[target_outcome].values

In [ ]:
print('Train: ', X_train.shape[0])
print('Val: ', X_val.shape[0])
print('Eval (internal validation): ', X_eval.shape[0])
print('Test: ', X_test.shape[0])
print('Test - Wales: ', X_testWales.shape[0])
print('Test - Scotland: ', X_testScotland.shape[0])

In [ ]:
pos_weight = trainingData[target_outcome].value_counts()[0]/trainingData[target_outcome].value_counts()[1]
neg_weight = trainingData[target_outcome].value_counts()[1]/trainingData[target_outcome].value_counts()[0]
class_weight = {0:1, 1:pos_weight}
print(class_weight)

In [ ]:
output_bias = np.array([np.log(neg_weight)])
output_bias = Constant(output_bias)

In [ ]:
valData.new_12MonthsOutcome.value_counts(normalize=False)

In [ ]:
from sklearn.utils import class_weight
sklearn_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight = dict(enumerate(sklearn_weights))

In [ ]:
# pickle.dump(code2idx_all_big, open('../SeqModel/code2idx_all_big.sav', 'wb'))
# pickle.dump(idx2code_all_big, open('../SeqModel/idx2code_all_big.sav', 'wb'))
# pickle.dump(data, open('../SeqModel/data_all_big.sav', 'wb'))

In [ ]:
%%time

# create the model
# embedding_vector_length = int(np.sqrt(vocab_size))
embedding_vector_length = 350
earlyStopping = EarlyStopping(monitor='val_auc', patience=10, verbose=0, mode='max', restore_best_weights=True)
mcp_save = ModelCheckpoint('../SeqModel/seqModel_all_new.mdl_wts.hdf5', save_best_only=True, monitor='val_auc', mode='min')
# class_weight = {0: 1, 1: 6}


with tf.device('/GPU:0'):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_codes))
    model.add(LSTM(128, return_sequences=True,
                   bias_regularizer=L1L2(l1=0.0, l2=0.01)))
    model.add(Dropout(0.5))
    model.add(LSTM(128, return_sequences=True,
                   bias_regularizer=L1L2(l1=0.0, l2=0.01)))
    model.add(Dropout(0.5))
    model.add(LSTM(128, bias_regularizer=L1L2(l1=0.0, l2=0.01)))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    # model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = SGD(learning_rate=5e-3, clipvalue=0.5)
    metrics = [
        AUC(num_thresholds=3, name='auc', curve='ROC'),
        tf.keras.metrics.Precision(name='prec'),
        tf.keras.metrics.Recall(name='rec'),
        tf.keras.metrics.TrueNegatives(name='TN'),
        tf.keras.metrics.TruePositives(name='TP'),
        tf.keras.metrics.PrecisionAtRecall(0.8)
    ]
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=metrics, )
    print(model.summary())
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                        epochs=1000, batch_size=256, 
                        class_weight = class_weight, 
                        callbacks = [earlyStopping, mcp_save, PlotLossesKeras()])



In [ ]:
model.summary()

In [ ]:
import io
out_v = io.open('../SeqModel/vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('../SeqModel/metadata.tsv', 'w', encoding='utf-8')
for index, word in enumerate(vocab):
  if (index == 0)|(index == 1)|(index == 2):
    continue  # skip 0,1,2, it's padding, satr and end
  vec = embeddings_vector[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")

out_v.close()
out_m.close()

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
# plt.ylim(0.55,1)
plt.title('model AUC')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
# plt.ylim(0.1, 1.15)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
with tf.device('/CPU:0'):
    # model.evaluate(X_train, y_train)
    model.evaluate(X_val, y_val, batch_size=300)
    model.evaluate(X_eval, y_eval, batch_size=300)
    model.evaluate(X_test, y_test, batch_size=300)
    model.evaluate(X_testWales, y_testWales, batch_size=300)
    model.evaluate(X_testScotland, y_testScotland, batch_size=300)

In [ ]:
#Model evaluation function
def summariseResult (testY, preds):
    tn, fp, fn, tp = confusion_matrix(testY, preds).ravel()
    specificity = tn / (tn+fp)
    sensitivity = tp / (tp+fn)
    ppv = 100*tp/(tp+fp)
    npv = 100*tn/(fn+tn)
    acc = accuracy_score(testY, preds)
    f1score = f1_score(testY, preds, average = 'binary')
    balanceacc = balanced_accuracy_score(testY, preds)
    fpr, tpr, thresholds = roc_curve(testY, preds, pos_label=1)
    aucscore = auc(fpr, tpr)
    # aucscore = roc_auc_score(testY, preds)
    auprc = average_precision_score(testY, preds)
    # plot_confusion_matrix(model, testX, testY, cmap='viridis')  
    return np.round(acc,4), np.round(specificity,4), np.round(sensitivity,4), np.round(aucscore,4), np.round(auprc,4), np.round(balanceacc,4), np.round(f1score,4), np.round(ppv,4), np.round(npv,4)

data_test_Xs = [X_eval, X_test, X_testWales, X_testScotland]
data_test_ys = [y_eval, y_test, y_testWales, y_testScotland]
for data_test_X, data_test_y in zip(data_test_Xs, data_test_ys):
    with tf.device('/CPU:0'):
        preds = model.predict(data_test_X)
    preds = [0 if pred <0.5 else 1 for pred in preds]
    print(summariseResult(data_test_y, np.squeeze(preds)))

In [ ]:
# model.save('../SeqModel/model_all.h5')

In [ ]:
extractor = tf.keras.Model(inputs=model.inputs, outputs=model.get_layer('embedding').output)
embeddings_vector = model.layers[0].get_weights()[0]

# `embeddings` has a shape of (num_vocab, embedding_dim) 

# `word_to_index` is a mapping (i.e. dict) from words to their index, e.g. `love`: 69
words_embeddings = {w:embeddings_vector[idx] for w, idx in code2idx.items()}

vocab = list(code2idx.keys())

In [ ]:
from sklearn.decomposition import PCA
pca_2dim = PCA(random_state=0).fit_transform(embeddings_vector)[:,:2]
pca_2dim = pd.DataFrame(pca_2dim, columns=['PC1', 'PC2'])
pca_2dim['code'] = vocab

In [ ]:
asthma_readcodes = pd.read_csv('../RCodes/ClinicalCodes/definite_asthma_read_codes.csv')[['readcode_new']]
ethnicity = pd.read_csv('../RCodes/ClinicalCodes/Ethinicity.csv')[['Read.Code']]
smoking = pd.read_csv('../RCodes/ClinicalCodes/SmokingReadcodes.csv')[['Read Code']]
ICS = pd.read_csv('../RCodes/ClinicalCodes/ICS_Asthma.csv')[['code_id']]

In [ ]:
asthma_readcodes.columns = ['ReadCode']
asthma_readcodes['label'] = 'asthma specific'
asthma_readcodes['label_num'] = 0
ethnicity.columns = ['ReadCode']
ethnicity['label'] = 'ethnicity'
ethnicity['label_num'] = '1'
smoking.columns = ['ReadCode']
smoking['label'] = 'smoking'
smoking['label_num'] = '2'
ICS.columns = ['ReadCode']
ICS['label'] = 'ICS'
ICS['label_num'] = '3'

In [ ]:
df = pd.concat([asthma_readcodes, ethnicity, smoking, ICS])

In [ ]:
pca_subset = pca_2dim[pca_2dim.code.isin(df.ReadCode.unique())]

In [ ]:
pca_subset = df.merge(pca_subset, how='right', left_on='ReadCode', right_on='code')

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(pca_subset.PC1.values,pca_subset.PC2.values, c=pca_subset.label_num.values)
plt.show()
